<a href="https://colab.research.google.com/github/junsukha/pose-estimation-demo/blob/main/pose_estimation_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Refer to https://www.analyticsvidhya.com/blog/2021/10/human-pose-estimation-using-machine-learning-in-python/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mediapipe
import mediapipe as mp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 31.5 MB 69.4 MB/s 


In [ ]:
import cv2
import time
import numpy as np
import pandas as pd
import os


In [ ]:
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils # for drawing keypoints
points = mpPose.PoseLandmark # Landmarks

In [ ]:
data = []
for p in points:
  x = str(p)[13:]
  data.append(x + "_x")
  data.append(x + "_y")
  data.append(x + "_z")
  data.append(x + "_vis")
data = pd.DataFrame(columns = data) # Empty dataset

# New Section

In [12]:
from google.colab.patches import cv2_imshow

In [15]:
path = "/content/drive/MyDrive/Colab Notebooks/cv-final-project/yoga-data-set/DATASET/TEST/plank" # enter dataset path
count = 0
for img in os.listdir(path):
  temp = []
  img = cv2.imread(path + "/" + img)
  imageWidth, imageHeight = img.shape[:2]
  imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  blackie = np.zeros(img.shape) # Blank image
  results = pose.process(imgRGB) # extract key points
  if results.pose_landmarks:
    mpDraw.draw_landmarks(blackie, results.pose_landmarks, mpPose.POSE_CONNECTIONS)
    landmarks = results.pose_landmarks.landmark
    for i,j in zip(points, landmarks):
      temp = temp + [j.x, j.y, j.z, j.visibility]
    
    data.loc[count] = temp
    count+=1

  # cv2_imshow(img)
  # cv2_imshow(blackie)  
  # cv2.waitKey(100)
data.to_csv("dataset3.csv")